<a href="https://colab.research.google.com/github/jooniorp/DACON_NLP/blob/main/train_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

# KoBERT 모델과 토크나이저 로드
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TR

In [ ]:
path = "/content/drive/MyDrive/Dacon Hasol/train.csv"
data = pd.read_csv(path)
dataset = Dataset.from_pandas(pd.DataFrame(data))

# 데이터 전처리 함수
def preprocess_function(examples):
    inputs = tokenizer(
        examples["question"],
        max_length=384,
        truncation=True,
        padding="max_length",
    )

    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i in range(len(answers)):
        answer = answers[i]
        answer_tokens = tokenizer.tokenize(answer)

        answer_start_index = inputs["input_ids"][i].index(tokenizer.cls_token_id) + 1
        start_positions.append(answer_start_index)
        end_positions.append(answer_start_index + len(answer_tokens) - 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# 데이터 전처리 적용
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [ ]:
!pip install transformers[torch]

In [ ]:
# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

In [ ]:
# 모델 훈련
trainer.train()

Epoch,Training Loss,Validation Loss
